In [1]:
import dpp_nets.my_torch as my_torch
import torch
import numpy as np
from dpp_nets.my_torch.controlvar import compute_alpha
import matplotlib.pyplot as plt
from dpp_nets.dpp.score_dpp import score_dpp
from dpp_nets.dpp.sample_dpp import sample_dpp

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)

network_params = {'emb_in': 300, 'emb_h': 200, 'emb_out': 100,
                  'pred_in': 150, 'pred_h': 100, 'pred_out': 1,
                  'set_size': 40} 
dtype = torch.DoubleTensor

train_iter_base = 200
batch_size = 10
train_iter = batch_size * train_iter_base
sample_iter = 1
alpha_iter = 0
lr = 1e-5
weight_decay = 0
reg_exp = 1
reg_var = 0

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def plot_moving_average(plot_dict, name, n=200):
    temp = {k: torch.mean(torch.stack(v)) for k,v in plot_dict.items()}
    x, y = zip(*sorted(temp.items()))
    x = np.array(x)
    y = np.array(y)
    x = moving_average(x, n)
    y = moving_average(y, n)
    plt.plot(x,y)
    plt.title("Loss (Moving Average)")
    plt.savefig(str(name) + ".pdf", format='pdf')
    plt.ylabel('MSE')
    plt.xlabel('Training Iteration')
    plt.show()

In [11]:
alpha_iter = 0
overwrite = 0

set_seed(10)
baseline_0 = my_torch.DPPRegressor(network_params, dtype)
baseline_0.sample()

set_seed(13)
baseline_0.train_with_baseline(train_iter, batch_size, sample_iter, alpha_iter, lr, weight_decay, reg_exp, reg_var, overwrite)
baseline_0.evaluate(1000)

Target sampled was:  16.0
Network predicted:  
-5.9872
[torch.DoubleTensor of size 1]

Resulting loss is:  Variable containing:
 483.4362
[torch.DoubleTensor of size 1]

Prediction was based on  29.0 observations.
50 796.5861127548313
100 239.2788081700037
150 208.81018547402184
200 55.72543358038603
250 66.1099483156616
300 130.33793027195446
350 14.562950269165652
400 56.193007269633355
450 84.82694916611531
500 607.8770635551147
550 150.72781117578438
600 22.44469375993785
650 251.52967046123425
700 294.5949695658181
750 112.89757389089877
800 122.7686876787658
850 109.58883333470588
900 56.38403193464665
950 165.6062930470007
1000 171.28667929995606
1050 94.21506251893155
1100 226.225990439803
1150 41.66172156771141
1200 0.03389976400682766
1250 0.4495520946673808
1300 13.150464967964435
1350 165.2348556213591
1400 250.48860056258908
1450 73.81784510821369
1500 11.67876868519249
1550 1294.8841410085734
1600 17.130285104165083
1650 234.3810139893902
1700 127.25106189478002
1750 187.

(Variable containing:
  92.3639
 [torch.DoubleTensor of size 1], 10.807999999999991, 0.14540608500850963)

In [13]:
baseline_0.train_with_baseline(10000, 25, sample_iter, alpha_iter, lr, weight_decay, reg_exp, reg_var, overwrite)
baseline_0.evaluate(1000)

50 52.79522174039645
100 21.35126694300043
150 58.39967646906045
200 15.267550300643592
250 3.2583358551815746
300 24.660927529775844
350 0.662600946473848
400 3.3466878381615097
450 41.03203585155349
500 326.56420610716464
550 18.550585839917733
600 32.39405275240102
650 16.315677850784642
700 0.20741902916884747
750 52.58444944164064
800 2.0734837018061865
850 3.81715228377719
900 4.3549821257039625
950 5.002407780466141
1000 26.490053245882006
1050 0.8135887464392372
1100 6.641271037539644
1150 1.0890614784792443
1200 7.4910730149967115
1250 38.923466487755945
1300 68.93545183421973
1350 36.832983575836764
1400 225.4013008995119
1450 0.8763666174202852
1500 89.45511195451496
1550 5.934380374832063
1600 44.461789961036274
1650 22.970752565038012
1700 23.797577687570257
1750 27.72513694418875
1800 17.43508817520251
1850 38.87984196340899
1900 19.739555881621417
1950 1.3703928770592638e-05
2000 32.147776033558
2050 21.641919080980312
2100 27.013861932089263
2150 5.118101262061443
2200 

(Variable containing:
  24.7899
 [torch.DoubleTensor of size 1], 10.499000000000004, 0.10159039721297153)

In [18]:
embd = baseline_0.embedding
L = embd.mm(embd.t())

In [24]:
(L.triu() - torch.diag(L.diag()))**2

Variable containing:
 0.0000e+00  2.1613e+08  2.1055e+08  ...   2.1013e+08  2.1305e+08  2.1276e+08
 0.0000e+00  0.0000e+00  2.1175e+08  ...   2.1120e+08  2.1409e+08  2.1382e+08
 0.0000e+00  0.0000e+00  0.0000e+00  ...   2.0610e+08  2.0860e+08  2.0841e+08
                ...                   ⋱                   ...                
 0.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  2.0810e+08  2.0800e+08
 0.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  2.1063e+08
 0.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  0.0000e+00
[torch.DoubleTensor of size 40x40]

In [ ]:
alpha_iter = 0
overwrite = 500

set_seed(10)
baseline_500 = my_torch.DPPRegressor(network_params, dtype)
baseline_500.sample()

set_seed(13)
baseline_500.train_with_baseline(train_iter, batch_size, sample_iter, alpha_iter, lr, weight_decay, reg_exp, reg_var, overwrite)
baseline_500.evaluate(1000)

In [10]:
alpha_iter = 5
overwrite = 0
reg_exp = 10
batch_size = 20
lr = 1e-6
reg_exp = 1000
set_seed(10)
model = my_torch.DPPRegressor(network_params, dtype)
model.sample()

set_seed(13)
model.train_with_baseline(train_iter, batch_size, sample_iter, alpha_iter, lr, weight_decay, reg_exp, reg_var, overwrite)
model.evaluate(1000)

Target sampled was:  16.0
Network predicted:  
-5.9872
[torch.DoubleTensor of size 1]

Resulting loss is:  Variable containing:
 483.4362
[torch.DoubleTensor of size 1]

Prediction was based on  29.0 observations.
50 973.3920678551772
100 227.45190482307635
150 147.95155133660802
200 124.26881464189738
250 283.4651091299662
300 620.5129520367464
350 148.81369215745565
400 960.1969730933067
450 99.7436633814684
500 773.663252644457
550 1116.5478589136314
600 690.5532429504051
650 13.750938957507614
700 499.1640456895175


RuntimeError:  Lapack Error gesvd : 39 superdiagonals failed to converge. at /Users/soumith/miniconda2/conda-bld/pytorch_1493757319118/work/torch/lib/TH/generic/THTensorLapack.c:461

In [ ]:
{k: torch.var(torch.stack(v)) for k,v in model.alpha_dict.items()}

In [8]:
model.embedding_dict

defaultdict(list, {0: [
                4.8480  -4.3272 -10.8919  ...   23.1331  14.8405  12.1135
                5.2175  -2.0754 -10.4327  ...   24.0910  13.4920  11.4243
                9.7257  -3.2879  -7.4566  ...   22.0761  13.0037   9.8163
                         ...               ⋱              ...            
                7.5173  -2.1404 -11.4161  ...   22.1864  13.4349   7.2865
                5.1276  -4.4917 -11.2502  ...   23.2731  15.2361  11.9807
                4.8324  -6.8326 -11.6128  ...   23.2857  14.0881  12.5782
              [torch.DoubleTensor of size 40x100]], 1: [
               12.0627 -10.5557 -12.4607  ...   -5.6206  -5.8615 -21.1330
                9.1800 -10.4528 -15.1267  ...   -7.6963  -6.5256 -16.7513
               10.6949 -11.6959 -16.8252  ...   -7.9929  -4.3959 -18.8589
                         ...               ⋱              ...            
               11.9784 -10.7600 -12.4586  ...   -5.4807  -6.1908 -21.0800
               12.5930  -8.2329

In [ ]:
embd = model.embedding.data.numpy()

In [ ]:
index = model.subset.data.diag().numpy()
index = index.astype(bool)

In [ ]:
subembd = embd[index]

In [ ]:
submatrix = subembd.dot(subembd.T)

In [ ]:
submatinv = np.linalg.inv(submatrix)

In [ ]:
np.linalg.matrix_rank(submatrix)

In [ ]:
torch.randn(1).clamp(-.1,1.5)

In [ ]:
for i in range(280):
    print(np.sum(np.isinf(model.alpha_dict[i][0].numpy())))

In [ ]:
plot_moving_average(baseline_0.loss_dict, "alpha0", 150)

In [ ]:
plot_moving_average(baseline_500.loss_dict, "alpha500", 150)

In [ ]:
plot_moving_average(model.loss_dict, "alpha_est", 10)

In [ ]:
alpha_iter = 5

set_seed(10)
model = my_torch.DPPRegressor(network_params, dtype)
model.sample()

set_seed(13)
model.train_with_baseline(train_iter, batch_size, sample_iter, alpha_iter, lr, weight_decay, reg_exp, reg_var)
model.evaluate(500)

In [ ]:
covar = compute_covar(torch.stack(model.a_score_dict[0]), torch.stack(model.a_reinforce_dict[0]))

In [ ]:
covar.squeeze(0)[24] / torch.var(torch.stack(model.a_score_dict[0]), dim=0).squeeze(0)[24]

In [ ]:
model.alpha_dict[0][0][24]

In [ ]:
compute_alpha(model.a_reinforce_dict[0], model.a_score_dict[0])[24]

In [ ]:
np.sort(model.alpha.numpy())[:,-1]

In [ ]:
np.where(np.isinf(model.alpha.numpy()))

In [ ]:
np.where((model.alpha.numpy())>1000)

In [ ]:
# Loss Dict
print("Loss Dict: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.loss_dict.items()},"\n")
# Pred Dict
print("Pred Dict: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.pred_dict.items()},"\n")
# Subset Sizes
print("Subset Dict: ", {k: torch.sum(torch.stack(v)) for k,v in baseline.subset_dict.items()},"\n")
# Weights in Prediction Layer
print("Prediction Dict: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.pred_w2_mean.items()},"\n")

In [ ]:
# Loss Dict
print("Loss Dict: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.loss_dict.items()},"\n")
# Pred Dict
print("Pred Dict: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.pred_dict.items()},"\n")
# Subset Sizes
print("Subset Dict: ", {k: torch.sum(torch.stack(v)) for k,v in baseline.subset_dict.items()},"\n")
# Weights in Prediction Layer
print("Weights: ", {k: torch.mean(torch.stack(v)) for k,v in baseline.pred_w2_mean.items()},"\n")

In [ ]:
baseline.pred_w1_max

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import spline

plt.plot(x,y)
plt.show()

In [ ]:
def plot_loss(plot_dict1, plot_dict2, smooth=0):

    plot_dict1 = {k: torch.stack(v).mean() for k, v in plot_dict1.items()}
    plot_dict2 = {k: torch.stack(v).mean() for k, v in plot_dict2.items()}
    
    if not smooth:        
        x, y = zip(*sorted(plot_dict1.items()))
        plt.plot(x, y,"blue")     
        x, y = zip(*sorted(plot_dict2.items()))
        plt.plot(x, y,"red")      
        plt.show()
        
    else:
        x = list(plot_dict1.keys())
        y = list(plot_dict1.values())
        x_sm = np.array(x)
        y_sm = np.array(y)
        x_smooth = np.linspace(x_sm.min(), x_sm.max(), smooth)
        y_smooth = spline(x, y, x_smooth)
        plt.plot(x_smooth, y_smooth,"blue")
        
        x = list(plot_dict2.keys())
        y = list(plot_dict2.values())
        x_sm = np.array(x)
        y_sm = np.array(y)
        x_smooth = np.linspace(x_sm.min(), x_sm.max(), smooth)
        y_smooth = spline(x, y, x_smooth)
        plt.plot(x_smooth, y_smooth,"red")
        
        plt.show()

In [ ]:
plot_loss(model.loss_dict, model.loss_dict, smooth=40)

In [ ]:
model.evaluate(1000)

In [ ]:
baseline = my_torch.DPPRegressor(network_params, dtype)

In [ ]:
baseline.evaluate(1000)

In [ ]:
def compute_covar(scores, reinforce_grads):
    scores = scores - torch.mean(scores, dim=0).expand_as(scores)
    reinforce_grads = reinforce_grads - torch.mean(reinforce_grads, dim=0).expand_as(reinforce_grads)
    cov_reinforce_score = torch.mean(reinforce_grads * scores, dim=0)
    return cov_reinforce_score

In [ ]:
# Doing analysis!

embd = baseline_0.embedding.data.numpy()
L = embd.dot(embd.T)
e, v = np.linalg.eig(L)
res_1 = sample_dpp(e, v, one_hot=True)
res_2 = sample_dpp(e, v, one_hot=True)

grad_1 = score_dpp(embd, res_1)
grad_2 = score_dpp(embd, res_2)
both = np.intersect1d(np.where(res_1), np.where(res_2))
none = np.intersect1d(np.where(res_1 == 0), np.where(res_2 == 0))
either = np.setdiff1d(np.arange(40), np.union1d(both, none))

assert len(res_1) == len(both) + len(none) + len(either)
grads = np.stack([grad_1, grad_2])
var = np.var(grads, axis=0)

In [ ]:
var_both = var[both]
var_either = var[either]
var_none = var[none]

In [ ]:
np.var(var_both)

In [ ]:
np.var(var_either)

In [ ]:
import torch

def compute_alpha(reinforce_grads, logprob_grads, mean=False, noflip=False, clip=False): 
    """
    reinforce_grads is expected to be a list of Tensors that hold the REINFORCE gradients
    logprob_grads is expected to be a list of Tensors that hold the SCORES
        
    """
    assert len(reinforce_grads) == len(logprob_grads)
    
    scores = torch.stack(logprob_grads)
    reinforce_grads = torch.stack(reinforce_grads)
    min_loss = torch.min(reinforce_grads / scores)
    scores = scores - torch.mean(scores, dim=0).expand_as(scores)
    reinforce_grads = reinforce_grads - torch.mean(reinforce_grads, dim=0).expand_as(reinforce_grads)
    
    assert reinforce_grads.size() == scores.size()
    
    score_var = torch.var(scores, dim=0)
    cov_reinforce_score = torch.mean(reinforce_grads * scores, dim=0)
    alpha = (cov_reinforce_score / score_var).squeeze(0)
    alpha[alpha != alpha] = 0
            
    return alpha

In [ ]:
A = torch.Tensor([np.inf])

In [ ]:
A == A

In [ ]:
B = torch.Tensor([np.nan])

In [ ]:
B == B

In [ ]:
A[0] = np.nan

In [ ]:
{k: np.sum(np.isinf(torch.stack(v).numpy())) for k,v in model.alpha_dict.items()}

In [ ]:
A = compute_alpha(model.a_reinforce_dict[0], model.a_score_dict[0])

In [ ]:
reinforce_grads

In [ ]:
reinforce_grads = torch.stack(model.a_reinforce_dict[0]).numpy()
scores = torch.stack(model.a_score_dict[0]).numpy()

In [ ]:
scores = scores - np.mean(scores, axis=0)
score_var = np.var(scores, axis=0)
reinforce_grads = reinforce_grads - np.mean(reinforce_grads, axis=0)
cov_reinforce_score = np.mean(reinforce_grads * scores, axis=0)

In [ ]:
cov_reinforce_score / score_var

In [ ]:
model.alpha_dict[0][0]

In [ ]:
def compute_alpha_np(reinforce_grads, logprob_grads):
    assert len(reinforce_grads) == len(logprob_grads)
    
    scores = torch.stack(logprob_grads).numpy()
    reinforce_grads = torch.stack(reinforce_grads).numpy()
    min_loss = np.min(reinforce_grads / scores)
    scores = scores - np.mean(scores, axis=0)
    reinforce_grads = reinforce_grads - np.mean(reinforce_grads, axis=0)
    
    assert reinforce_grads.shape == scores.shape
    
    score_var = np.var(scores, axis=0)
    cov_reinforce_score = np.mean(reinforce_grads * scores, axis=0)
    alpha = (cov_reinforce_score / score_var)
    alpha[alpha != alpha] = 0
    
    return score_var

In [ ]:
import torch
import numpy as np

def compute_alpha(reinforce_grads, logprob_grads, mean=False, noflip=False, clip=False): 
    """
    reinforce_grads is expected to be a list of Tensors that hold the REINFORCE gradients
    logprob_grads is expected to be a list of Tensors that hold the SCORES
        
    """
    assert len(reinforce_grads) == len(logprob_grads)
    
    scores = torch.stack(logprob_grads)
    reinforce_grads = torch.stack(reinforce_grads)
    min_loss = torch.min(reinforce_grads / scores)
    scores = scores - torch.mean(scores, dim=0).expand_as(scores)
    reinforce_grads = reinforce_grads - torch.mean(reinforce_grads, dim=0).expand_as(reinforce_grads)
    
    assert reinforce_grads.size() == scores.size()
    
    score_var = torch.var(scores, dim=0)
    cov_reinforce_score = torch.mean(reinforce_grads * scores, dim=0)
    
    cov_reinforce_score[cov_reinforce_score == np.inf] = np.nan
    score_var[score_var == 0] = np.nan
    
    alpha = (cov_reinforce_score / score_var).squeeze(0)
    alpha[alpha != alpha] = 0
            
    if mean:
        alpha = torch.DoubleTensor([alpha.mean()]).expand_as(alpha)
    
    if noflip:
        alpha.clamp_(max=min_loss)
        
    if clip: 
        alpha.clamp_(min=-min_loss)
    
    return alpha


In [ ]:
np_res = compute_alpha_np(model.a_reinforce_dict[0], model.a_score_dict[0])

In [ ]:
torch_res = compute_alpha(model.a_reinforce_dict[0], model.a_score_dict[0])

In [ ]:
np_res - torch_res.numpy()

In [ ]:
np.sum(np.isinf(torch_res.numpy()))